# WEB MINING PROJECT:
# Mining Information from Social Media Networks During Crisis Events

First, let's import the required packages:

In [4]:
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Second, let's upload the graph we will be working, the *database_formated_for_NetworkX.graphml* being in the current working directory:

In [2]:
path = 'data/database_formated_for_NetworkX.graphml'
graph = nx.read_graphml(path)

## Preliminary Analyses:

print('Number of nodes:', graph.number_of_nodes())
print('Number of edges:', graph.number_of_edges())
print('Directed graph:', nx.is_directed(graph))
print('Weighted graph:', any('weight' in data for _, _, data in graph.edges(data=True)))

Our multi-layered graph is directed, unweighted and comprised of 109,627 nodes and 311,654 edges

Let's look closer to the structure of the nodes and the edges:

In [ ]:
#About the nodes:

i = 0
for node, data in graph.nodes(data=True):
    i+=1
    print("Node:", node)
    for key, value in data.items():
        print("\t", key, ":", value)
    if i > 10:
        break

In [ ]:
#About the edges:

i = 0
for edge in graph.edges(data=True):
    i+=1
    print("Edge:", edge)
    if i > 10:
        break

Let's see what node labels and event types we have:

In [ ]:
labels_values = set()
event_type_values = set()

for node, data in graph.nodes(data=True):
    labels_values.update(data.get('labels', '').split(":"))
    event_type_values.add(data.get('eventType', ''))
    
print(labels_values)
print(event_type_values)

We have nodes of type 'Tweet', 'Event', 'User', 'Hashtag' and 'PostCategory'. We also have 6 seperate events to consider, 'wildfire', 'earthquake', 'shooting', 'typhoon', 'flood' and 'bombing', which will be the attributes behind the construction of our subgraphs.

## Separation of our graph into event-based subgraphs

In [6]:
subgraphs = {event_type: graph.subgraph([node for node, data in graph.nodes(data=True) if data.get('eventType') == event_type]) 
             for event_type in ['wildfire', 'earthquake', 'typhoon', 'bombing', 'flood', 'shooting']}

In [ ]:
# Creating a dictionary with each event as keys, and id values linking tweets to events as values

event_trecisid_mapping = {}
events = ['wildfire', 'earthquake', 'typhoon', 'bombing', 'flood', 'shooting']

for event_type in events:
    trecisid_values = []
    
    #Iterate over the nodes of the different event type and collect their 'trecisid' values
    for node, data in subgraphs[event_type].nodes(data=True):
        if 'trecisid' in data:
            trecisid_values.append(data['trecisid'])
    
    #Add the event type and its corresponding 'trecisid' values to the dictionary
    event_trecisid_mapping[event_type] = trecisid_values
    
event_trecisid_mapping

In [ ]:
# Creating a dictionary of events and their corresponding 'Tweet' nodes:

event_tweets = {}

for event_type in events:
    event_trecisids = event_trecisid_mapping.get(event_type, set())
    
    event_tweet_list = []
    
    for node, data in graph.nodes(data=True):
        if 'topic' in data:
            if data['topic'] in event_trecisids:
                event_tweet_list.append(node)
    
    event_tweets[event_type] = event_tweet_list

print(event_tweets['wildfire'][:10])
print(event_tweets['bombing'][:10])

In [9]:
# Creating a function that creates the subgraphs we want based on the nodes in event_tweets and the nodes they share edges with in the initial graph

def create_subgraph(event, graph):
    nodes = event_tweets.get(event, [])
    node_list = []
    
    for edge in graph.edges():

        if edge[0] in nodes or edge[1] in nodes:
            node_list.append(edge[0])
            node_list.append(edge[1])

    node_list = list(dict.fromkeys(node_list))

    subgraph = graph.subgraph(node_list)
    subgraph_name = f"{event}_subgraph"
    globals()[subgraph_name] = subgraph

In [10]:
# Calling the function for all event types

for event in events:
    create_subgraph(event, graph)

In [ ]:
# Example: the nodes of the earthquake subgraph
i=0
for node, data in wildfire_subgraph.nodes(data=True):
    i+=1
    print("Node:", node)
    for key, value in data.items():
        print("\t", key, ":", value)
    if i>10:
        break

In [ ]:
# Example: the nodes of the wildfire subgraph
i = 0
for edge in wildfire_subgraph.edges(data=True):
    i+=1
    print("Edge:", edge)
    if i>10:
        break

In [ ]:
len(wildfire_subgraph.edges()) + len(earthquake_subgraph.edges()) + len(shooting_subgraph.edges()) + len(bombing_subgraph.edges())+ len(flood_subgraph.edges())+ len(typhoon_subgraph.edges())

In [ ]:
len(wildfire_subgraph.nodes()) + len(earthquake_subgraph.nodes()) + len(shooting_subgraph.nodes()) + len(bombing_subgraph.nodes())+ len(flood_subgraph.nodes())+ len(typhoon_subgraph.nodes())

In [15]:
subgraphs = {'wildfire_subgraph': wildfire_subgraph,
             'earthquake_subgraph': earthquake_subgraph,
             'typhoon_subgraph': typhoon_subgraph,
             'bombing_subgraph': bombing_subgraph,
             'flood_subgraph': flood_subgraph,
             'shooting_subgraph': shooting_subgraph}

We now have 6 seperate subgraphs of our original graph that represent each major event category. For conveniency, We have stored all our subgraphs in a subgraph dictionary called "subgraphs".

## 1.1 The Intensity of the Social Activity

### 1.1.1 Total number of tweets per subgraph

In [ ]:
total_tweets_dict = {}

for subgraph_name, subgraph in subgraphs.items():
    total_tweets = sum(1 for node, data in subgraph.nodes(data=True) if data['labels'] == ':Tweet')
    total_tweets_dict[subgraph_name] = total_tweets

    
print("Total tweets in each subgraph:")
for subgraph_name, total_tweets in total_tweets_dict.items():
    print(f"{subgraph_name}: {total_tweets}")

There are more tweets in the typhoon subgraph (13,981) than any other subgraph. This is followed by the earthquake subgraph (13,056), the shooting subgraph (9,133), the wildfire subgraph (7826) and finally the bombing subgraph (4,186). Twitter users seem to be more inclined to tweet about a typhoon than any over event.

In [ ]:
# Extract subgraph names and total tweet counts from the dictionary
subgraph_names = list(total_tweets_dict.keys())
total_tweets_counts = list(total_tweets_dict.values())

# Define colors for each subgraph
colors = ['orange', 'brown', 'lightblue', 'red', 'blue', 'black']

# Plot the bar graph with specified colors
plt.figure(figsize=(10, 6))
plt.bar(subgraph_names, total_tweets_counts, color=colors)

# Add labels and title
plt.xlabel('Event Subgraph')
plt.ylabel('Total Number of Tweets')
plt.title('Total Tweets in Each Subgraph')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
total_tweets_dict = {}

for subgraph_name, subgraph in subgraphs.items():
    # Initialize counters for tweets and retweets
    total_tweets = 0
    total_retweets = 0
    
    # Iterate over nodes to count tweets
    for node, data in subgraph.nodes(data=True):
        if data.get('labels') == ':Tweet':
            total_tweets += 1
            
    # Iterate over edges to count retweets
    for source, target, data in subgraph.edges(data=True):
        if data.get('label') == 'RETWEETS':
            total_retweets += data.get('times', 0)
    
    # Update total_tweets_dict with total counts
    total_tweets_dict[subgraph_name] = total_tweets + total_retweets

# Print the total tweets (including retweets) in each subgraph
print("Total tweets (including retweets) in each subgraph:")
for subgraph_name, total_tweets in total_tweets_dict.items():
    print(f"{subgraph_name}: {total_tweets}")

If we include retweets in our tweet count, the ranking established above remains unchanged. 

In [ ]:
# Extract subgraph names and total tweet counts from the dictionary
subgraph_names = list(total_tweets_dict.keys())
total_tweets_counts = list(total_tweets_dict.values())

# Define colors for each subgraph
colors = ['orange', 'brown', 'lightblue', 'red', 'blue', 'black']

# Plot the bar graph with specified colors
plt.figure(figsize=(10, 6))
plt.bar(subgraph_names, total_tweets_counts, color=colors)

# Add labels and title
plt.xlabel('Event Subgraph')
plt.ylabel('Total Number of Tweets')
plt.title('Total Tweets (including retweets) in Each Subgraph')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Display the plot
plt.tight_layout()
plt.show()

### 1.1.2 Total followers reached by tweets per subgraph

In [ ]:
total_followers_dict = {}

for subgraph_name, subgraph in subgraphs.items():
    total_followers = 0
    for tweet_node, user_node, data in subgraph.edges(data=True):
        if subgraph.nodes[user_node]['labels'] == ':User':
            followers_count = subgraph.nodes[user_node].get('followers_count', 0)
            total_followers += followers_count
    total_followers_dict[subgraph_name] = total_followers

print("Total followers reached by tweets in each subgraph:")
for subgraph_name, total_followers in total_followers_dict.items():
    print(f"{subgraph_name}: {total_followers}")
#doesn't consider overlap between followers of some users and followers of others.

We notice that tweets in the earthquake subgraph tend to reach the most followers, followed by tweets in the shooting subgraph, the typhoon subgraph, the bombing subgraph, the flood subgraph and finally the wildfire subgraph

In [ ]:
subgraph_names = list(total_tweets_dict.keys())
total_tweets_counts = list(total_followers_dict.values())

# Define colors for each subgraph
colors = ['orange', 'brown', 'lightblue', 'red', 'blue', 'black']

# Plot the bar graph with specified colors
plt.figure(figsize=(10, 6))
plt.bar(subgraph_names, total_tweets_counts, color=colors)

# Add labels and title
plt.xlabel('Event Subgraph')
plt.ylabel('Total Number of Tweets')
plt.title('Total followers reached by tweets in each subgraph')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Display the plot
plt.tight_layout()
plt.show()

### 1.1.3 Hashtags per subgraph

In [ ]:
hashtags = []

# Iterate over nodes in the subgraph
for node, data in graph.nodes(data=True):
    # Check if the node has labels ":Hashtag"
    if data.get('labels') == ':Hashtag':
        # Append the hashtag ID to the list
        hashtags.append(data['id'])

print(len(hashtags))
print(len(set(hashtags)))

# exemples
hashtags[:10]

There is 10,441 different subgraphs in total

In [ ]:
from collections import Counter

# Initialize an empty dictionary to store DataFrames for each subgraph
hashtag_dfs = {}

# Iterate over each subgraph
for subgraph_name, subgraph in subgraphs.items():
    # Initialize an empty list to store all hashtag words
    hashtag_words = []

    # Iterate over nodes in the subgraph
    for node, data in subgraph.nodes(data=True):
        # Check if the node has labels ":Tweet"
        if data.get('labels') == ':Tweet':
            # Get the text of the tweet
            tweet_text = data.get('text', '')
            # Split the text into words
            words = tweet_text.split()
            # Extract words starting with "#"
            hashtag_words.extend([word.strip("#") for word in words if word.startswith("#")])

    # Convert the list of hashtag words into a Counter object to count occurrences
    hashtag_counts = Counter(hashtag_words)

    # Convert the Counter object into a pandas DataFrame
    hashtag_df = pd.DataFrame(hashtag_counts.items(), columns=['Hashtag', 'Occurences'])

    # Sort the DataFrame by the number of occurrences in descending order
    hashtag_df = hashtag_df.sort_values(by='Occurences', ascending=False)

    # Reset the index of the DataFrame
    hashtag_df = hashtag_df.reset_index(drop=True)

    # Store the DataFrame in the dictionary with the subgraph name as the key
    hashtag_dfs[subgraph_name] = hashtag_df

# Print the DataFrames for each subgraph
for subgraph_name, df in hashtag_dfs.items():
    print(f"Exemples of Hashtags for {subgraph_name} subgraph:")
    print(df[:4])

In [ ]:
wildfire_hashtags_df = hashtag_dfs['wildfire_subgraph']
wildfire_hashtags_df['Subgraph'] = "Wildfire"

filtered_hashtag_df = wildfire_hashtags_df[wildfire_hashtags_df['Occurences'] >= 20]

# Plot the filtered DataFrame as a histogram
plt.figure(figsize=(10, 6))
plt.bar(filtered_hashtag_df['Hashtag'], filtered_hashtag_df['Occurences'], color='skyblue')
plt.xlabel('Hashtag')
plt.ylabel('Occurences')
plt.title('Hashtag Occurrences in the Wildfire Subgraph (Count >= 20)')
plt.xticks(rotation=90)  
plt.tight_layout()  
plt.show()

The hashtag "ymmfire" was by far the most used hashtag in the wildfire subgraph with 2,408 total occurences

In [ ]:
earthquake_hashtags_df = hashtag_dfs['earthquake_subgraph']
earthquake_hashtags_df['Subgraph'] = "Earthquake"

filtered_hashtag_df = earthquake_hashtags_df[earthquake_hashtags_df['Occurences'] >= 20]

# Plot the filtered DataFrame as a histogram
plt.figure(figsize=(10, 6))
plt.bar(filtered_hashtag_df['Hashtag'], filtered_hashtag_df['Occurences'], color='skyblue')
plt.xlabel('Hashtag')
plt.ylabel('Occurences')
plt.title('Hashtag Occurrences in the Earthquake Subgraph (Count >= 20)')
plt.xticks(rotation=90) 
plt.tight_layout()  
plt.show()

The hashtag "NepalEarthquake" was by far the most used hashtag in the earthquake subgraph with 1,154 total occurences

In [ ]:
typhoon_hashtags_df = hashtag_dfs['typhoon_subgraph']
typhoon_hashtags_df['Subgraph'] = "Typhoon"

filtered_hashtag_df = typhoon_hashtags_df[typhoon_hashtags_df['Occurences'] >= 20]

# Plot the filtered DataFrame as a histogram
plt.figure(figsize=(10, 6))
plt.bar(filtered_hashtag_df['Hashtag'], filtered_hashtag_df['Occurences'], color='skyblue')
plt.xlabel('Hashtag')
plt.ylabel('Occurences')
plt.title('Hashtag Occurrences in the Typhoon Subgraph (Count >= 20)')
plt.xticks(rotation=90) 
plt.tight_layout()  
plt.show()

The hashtag "RubyPH" was by far the most used hashtag in the typhoon subgraph with 1,791 total occurences

In [ ]:
bombing_hashtags_df = hashtag_dfs['bombing_subgraph']
bombing_hashtags_df['Subgraph'] = "Bombing" 

filtered_hashtag_df = bombing_hashtags_df[bombing_hashtags_df['Occurences'] >= 20]

# Plot the filtered DataFrame as a histogram
plt.figure(figsize=(10, 6))
plt.bar(filtered_hashtag_df['Hashtag'], filtered_hashtag_df['Occurences'], color='skyblue')
plt.xlabel('Hashtag')
plt.ylabel('Occurences')
plt.title('Hashtag Occurrences in the Bombing Subgraph (Count >= 20)')
plt.xticks(rotation=90)  
plt.tight_layout()  
plt.show()

The hashtag "PrayForParis" was by far the most used hashtag in the bombing subgraph with 325 total occurences

In [ ]:
flood_hashtags_df = hashtag_dfs['flood_subgraph']
flood_hashtags_df['Subgraph'] = "Flood"

filtered_hashtag_df = flood_hashtags_df[flood_hashtags_df['Occurences'] >= 20]

# Plot the filtered DataFrame as a histogram
plt.figure(figsize=(10, 6))
plt.bar(filtered_hashtag_df['Hashtag'], filtered_hashtag_df['Occurences'], color='skyblue')
plt.xlabel('Hashtag')
plt.ylabel('Occurences')
plt.title('Hashtag Occurrences in the Flood Subgraph (Count >= 20)')
plt.xticks(rotation=90) 
plt.tight_layout()  
plt.show()

The hashtag "yycflood" was by far the most used hashtag in the flood subgraph with 533 total occurences

In [ ]:
shooting_hashtags_df = hashtag_dfs['shooting_subgraph']
shooting_hashtags_df['Subgraph'] = "Shooting"

filtered_hashtag_df = shooting_hashtags_df[shooting_hashtags_df['Occurences'] >= 20]

# Plot the filtered DataFrame as a histogram
plt.figure(figsize=(10, 6))
plt.bar(filtered_hashtag_df['Hashtag'], filtered_hashtag_df['Occurences'], color='skyblue')
plt.xlabel('Hashtag')
plt.ylabel('Occurences')
plt.title('Hashtag Occurrences in the Shooting Subgraph (Count >= 20)')
plt.xticks(rotation=90)  
plt.tight_layout()  
plt.show()

The hashtag "Dallas" was by far the most used hashtag in the shooting subgraph with 493 total occurences.

In [ ]:
hashtags_dfs = [wildfire_hashtags_df, earthquake_hashtags_df, typhoon_hashtags_df, bombing_hashtags_df, flood_hashtags_df, shooting_hashtags_df] 

# Concatenate the DataFrames along rows
combined_hashtags_dfs = pd.concat(hashtags_dfs, ignore_index=True)

combined_hashtags_dfs_sorted = combined_hashtags_dfs.sort_values(by='Occurences', ascending=False)

# The top 25 rows
top_hashtags = combined_hashtags_dfs_sorted.head(25)

# Histogram
plt.figure(figsize=(10, 6))
sns.barplot(x='Hashtag', y='Occurences', hue='Subgraph', data=top_hashtags, palette=colors)
plt.title('Top 25 Hashtags')
plt.xlabel('Hashtag')
plt.ylabel('Occurences')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## 1.2 The Central Users Based on Different Criteria

In a Twitter network, different centrality measures can provide insights into a user's ability to connect with others, spread information, and gather information. Here are centrality measures relevant to each of these aspects:

**Connecting Users:**

*Betweenness Centrality*: Users with high betweenness centrality act as bridges between different parts of the network, facilitating communication and connections between otherwise disconnected users or communities. They play a crucial role in connecting users and fostering network cohesion.

Note that the calling of the beetweenness centrality function takes time, and that the code chunks can be long to run. Do not hesitate to comment the *nx.betweenness_centrality* function in the following function, that will highly speed up computation.

**Spreading Information:**

*Degree Centrality*: Users with high degree centrality are connected to a large number of other users through interactions such as mentions, retweets, replies, and follows. They have the potential to reach a wide audience and play a significant role in spreading information within the network.

**Gathering Information:**

*Closeness Centrality*: Users with high closeness centrality are close to other users in terms of shortest path distances, allowing them to quickly access information from a wide range of sources within the network. They are well-positioned to gather information efficiently and maintain awareness of network dynamics.


In the following, we will only consider the user nodes of our network and calculate for each of our event-based subgraphs the top 5 users with the highest betweenness centrality, degree centrality and closeness centrality.

In [31]:
# let us first define a function to compute the centralities measures of a given subgraph
def centrality_function(graph):
    # Filter nodes to only include user nodes
    filtered_nodes = [node for node, data in graph.nodes(data=True) if ':User' in data.get('labels', '') or ':Tweet' in data.get('labels', '')]

    # Create a subgraph containing only user nodes
    user_subgraph = graph.subgraph(filtered_nodes)

    # Calculate centralities for user nodes
    centralities = {
        'betweenness_centrality': nx.betweenness_centrality(user_subgraph),
        'degree_centrality': nx.degree_centrality(user_subgraph),
        'closeness_centrality': nx.closeness_centrality(user_subgraph)
    }

    # Identify top 3 users with the highest centralities per centrality measure
    top_users_by_centrality = {}

    for centrality_measure, centrality_dict in centralities.items():
        top_users_by_centrality[centrality_measure] = sorted(centrality_dict.items(), key=lambda x: x[1], reverse=True)[:3]

    # Print top 3 users with the highest centralities and all node attributes
    for centrality_measure, top_users in top_users_by_centrality.items():
        print(f"Top 3 users by {centrality_measure}:")
        for user_id, centrality_score in top_users:
            node_data = user_subgraph.nodes[user_id]
            print(f"\tUser {user_id} - Centrality: {centrality_score}")
            for key, value in node_data.items():
                print(f"\t\t{key}: {value}")
        print("")

### 1.2.1 Wildfire subgraph

In [ ]:
centrality_function(wildfire_subgraph)

In the wildfire subgraph:

- User node n87002, who goes by the screen name "yulsman" and has 1,897 followers has the highest betweeness centrality. They are the best at connecting users.

- User node n88567, who goes by the screen name "BreakingNews" and has 9,345,296 followers has the highest degree centrality. They are the best at spreading information. 

- User node n64463, who goes by the screen name "AP" (The Associated Press) and has 12,790,776 followers has the highest closeness centrality. They are the best at gathering information. 

For each centrality, the top 3 is listed above, with all their respective attributes, if one wants to gain more insight on other central users within our twitter network for each centrality.

### 1.2.2 Earthquake Subgraph

In [ ]:
centrality_function(earthquake_subgraph)

In the earthquake subgraph:

- User node n88567, who goes by the screen name "BreakingNews" and has 9,345,296 followers has the highest betweeness centrality. They are the best at connecting users.

- User node n90039, who goes by the screen name "NewEarthquake" and has 891,454 followers has the highest degree centrality. They are the best at spreading information. 

- User node n96126, who goes by the screen name "narendramodi" (the prime minister of India) and has 43,346,394 followers has the highest closeness centrality. They are the best at gathering information. 

For each centrality, the top 3 is listed above, with all their respective attributes, if one wants to gain more insight on other central users within our twitter network for each centrality.

### 1.2.3 Shooting Subgraph

In [ ]:
centrality_function(shooting_subgraph)

In the shooting subgraph:

- User node n83848, who goes by the screen name "CNN" and has 41,613,138 followers has the highest betweeness centrality. They are the best at connecting users.

- User node n96724, who goes by the screen name "realDonaldTrump" (the previous Uninted States president) and has 48,268,973 followers has the highest degree centrality. They are the best at spreading information. 

- User node n96724 (again), who goes by the screen name "realDonaldTrump" and has 48,268,973 followers has the highest closeness centrality. They are the best at gathering information. 

For each centrality, the top 3 is listed above, with all their respective attributes, if one wants to gain more insight on other central users within our twitter network for each centrality.

### 1.2.4 Bombing Subgraph

In [ ]:
centrality_function(bombing_subgraph)

In the bombing subgraph:

- User node n88246, who goes by the screen name "cnnbrk" (CNN Breaking News) and has 55,261,111 followers has the highest betweeness centrality. They are the best at connecting users.

- User node n64463, who goes by the screen name "AP" (The Associated Press) and has 12,790,776 followers has the highest degree centrality. They are the best at spreading information. 

- User node n64463 (again), who goes by the screen name "AP" (The Associated Press) and has 12,790,776 followers has the highest closeness centrality. They are the best at gathering information. 

For each centrality, the top 3 is listed above, with all their respective attributes, if one wants to gain more insight on other central users within our twitter network for each centrality.

### 1.2.5 Flood Subgraph

In [ ]:
centrality_function(flood_subgraph)

In the flood subgraph:

- User node n98043, who goes by the screen name "nenshi" and has 376,529 followers has the highest betweeness centrality. They are the best at connecting users.

- User node n58501, who goes by the screen name "ANCALERTS" (ABS-CBN News Channel) and has 4,426,900 followers has the highest degree centrality. They are the best at spreading information. 

- User node n94255, who goes by the screen name "CyrilRamaphosa" (Cyril Ramaphosa, the president of South Africa) and has 569,816 followers has the highest closeness centrality. They are the best at gathering information. 

For each centrality, the top 3 is listed above, with all their respective attributes, if one wants to gain more insight on other central users within our twitter network for each centrality.

### 1.2.6 Typhoon Subgraph

In [ ]:
centrality_function(typhoon_subgraph)

In the typhoon subgraph:

- User node n58501, who goes by the screen name "ANCALERTS" (ABS-CBN News Channel) and has 4,426,900 followers has the highest betweeness centrality. They are the best at connecting users.

- User node n58501 (again), who goes by the screen name "ANCALERTS" (ABS-CBN News Channel) and has 4,426,900 followers has the highest degree centrality. They are the best at spreading information. 

- User node n58317, who goes by the screen name "dost_pagasa" and has 5,619,485 followers has the highest closeness centrality. They are the best at gathering information. 

For each centrality, the top 3 is listed above, with all their respective attributes, if one wants to gain more insight on other central users within our twitter network for each centrality.

We notice, for all subgraphs, that news outlets and the press such as CNN, ABC news, The Asscociated Press, often have the highest betweeness and/or degree centrality. They are the best at connecting users and spreading information. Given their nature, this make a lot of sence. Furthermore, we notice that influencers and polititians often have the highest closeness centrality and are the best at gathering information. 

## 1.3 The Distributions of the Tweet Posts and their Main Topics w.r.t Levels of Criticality Among the “Low”, “Medium”, “High” and “Critical”

We will here analyze the different kinds of events in the twitter network related to different crises. 

There are 6 different types of events :

- *Bombings*
- *Earthquakes*
- *Floods*
- *Shootings*
- *Typhoons*
- *Wildfires*

(Note: To view our analysis for each subgraph, one must change the variable 'chosen_event' below, and run the subsequent code)

In [38]:
chosen_event = 'bombing' #can modify this variable to 'wildfire', 'earthquake', 'flood', 'shooting', 'typhoon' or 'bombing'
g = subgraphs.get(chosen_event + '_subgraph')

First, let us extract all the tweets from the 'chosen_event' events

In [ ]:
tweets = []
for node, data in g.nodes(data=True):
    for key, value in data.items():
        if key == 'labels':
            if value == ':Tweet':
                tweets.append({node : data})

print(f"There are {len(tweets)} tweets for the {chosen_event} event type\n")
print("Example of tweet :")
print(tweets[0])

# we remove unknown criticality tweets
tweets = [tweet for tweet in tweets if tweet[list(tweet.keys())[0]]['annotation_postPriority'] != 'Unknown']

Then, we extract the events names and add them to the tweets dictionnary for better visualisation 

In [ ]:
events = []
for node, data in g.nodes(data=True):
    for key, value in data.items():
        if key == 'labels':
            if value == ':Event':
                events.append({node : data})

events = [list(dict(events[i].items()).values())[0] for i in range(len(events))]
events

In [ ]:
for i in range(len(tweets)) : 
    event_id = list(dict(tweets[i].items()).values())[0]['topic']
    for item in events:
        if item.get('trecisid') == event_id:
            desired_event = item

    for inner_dict in tweets[i].values():
        inner_dict['ref_event'] = desired_event['id']

# we have the reference event at the end
tweets[0]

We can see that for each of the tweets, we have added an attribute "ref_event" to their attribute dictionnary. This represents the name of the referenced event in the tweet.

## 1.3.1 Criticality of the tweets count

In [42]:
# first, we convert the list of dictionnaries into a pandas dataframe which will be easier to manipulate
list_of_tweets = [list(dict(tweets[i].items()).values())[0] for i in range(len(tweets))]
df_tweets = pd.DataFrame(list_of_tweets)
df_tweets.head()

# visual purposes
order_crit = ['Critical', 'High', 'Medium', 'Low']
palette = sns.color_palette("coolwarm")
palette = palette[::-1]

Let us visualize the distribution of the criticality of the tweets:

In [ ]:
df_tweets['annotation_postPriority'].value_counts()

In [ ]:
plt.figure(figsize=(8, 3))
sns.countplot(y=df_tweets['annotation_postPriority'], color='lightblue', order=order_crit)

We notice, for all event-types, that:
- low criticality tweets are more common than medium centrality tweets
- medium criticality tweets are more common than high centrality tweets
- high criticality tweets are more common than critical centrality tweets

### 1.3.2 Tweet count for each sub-event, separated by level of criticality
(Note: we define a sub-event as a smaller event of a bigger general event type, for example the colorado fire is a sub-event of event 'wildfire')

In [ ]:
ax = sns.countplot(data=df_tweets, 
                   x='ref_event', 
                   hue='annotation_postPriority', 
                   palette=palette, 
                   hue_order=order_crit)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

plt.show()

- For all event types:

We observe that the majority of tweets are of low priority generally speaking. They usually represent the majority of tweets within sub-events too. However, we still notice some high and critical tweets, which will be interesting to study. An exception arrises when studying the flood and shooting event types, where for the South Africa flood of 2019 and San Diego shooting of 2019, we have high and medium priority tweets respectively that are far more common. 

### 1.3.3 Social activity associated with the tweets by level of criticality

In [ ]:
plt.figure(figsize = (15, 5))
sns.violinplot(data = df_tweets,
               x = 'annotation_postPriority',
               y = 'retweet_count',
               order = order_crit,
               color = 'lightblue')
plt.ylim(0, 1000)

- For wildfire event type: 
We observe that while there are numerous outliers with very high retweet counts, likely attributed to tweets from influential or famous users, the overall trend indicates that as tweet criticality increases, tweets tend to receive more retweets on average. This suggests active engagement from the Twitter community in amplifying important information.

- For earthquake event type:
We observe that while there are numerous outliers with high, medium and low retweet counts, likely attributed to tweets from influential or famous users, we observe that critical tweets receive far more retweets on average compared to other criticality types. This is followed by medium and low tweets and finally high criticality tweets who appear to receive far less retweets on average. This suggests active engagement from the Twitter community in amplifying critical information more than anything else.

- For flood and shooting event types:
We observe numerous outliers with high, medium and low retweet counts. Little outliers for critical priority tweets. Contrary to the two previous event types analysed, the critical, high and medium priority tweets appear to have far less average retweets than low priority tweets. This suggests active engagement from the Twitter community in amplifying low priority information more than anything else.

- For typhoon event type: 
We observe many outliers for all the tweet prioprity types. Medium criticality tweets seem to be far more retweeted on average than all the rest. The average retweets seem to be comparable for the other criticality types. This suggests active engagement from the Twitter community in amplifying medium priority information more than anything else.

- For bombing event type: 
We observe little outliers all around. There are far more average retweet counts than the rest of the event types. High criticality tweets seem to be the tweets with the most average retweets by far, followed by medium criticality tweets. This suggests active engagement from the Twitter community in amplifying medium priority information more than anything else.

This can be confirmed by looking at the mean number of retweets by tweet priority :

In [ ]:
# we can also look at the mean number of retweets for each level of criticality
df_tweets[['annotation_postPriority', 'retweet_count']].groupby('annotation_postPriority').mean()

### 1.3.4 Which events contain the more critical tweets and which are the most retweeted

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.boxplot(data=df_tweets[df_tweets['retweet_count'] < 500], 
                 x='ref_event', 
                 y='retweet_count', 
                 hue='annotation_postPriority', 
                 showfliers=False,
                 palette=palette,
                 hue_order=order_crit)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

plt.show()

- For wildfire event-type:
The result found in 1.3.3 is confirmed here when we separate the tweets by sub-event. We clearly see the higher values are associated with the higher priority level of tweets in most cases.

- For earthquake, flood, shooting and typhoon event-types:
The result found in 1.3.3 is not necessarily confirmed here when we separate the tweets by sub-event. We notice that the prevelence of retweets by post priority changes from one sub-event to the next.

- For bombing event-types:
We notice that the prevelence of retweets by post priority does not change from one sub-event to the next. 

### 1.3.5 What are the 'favourite counts' for each level of criticality

In [ ]:
plt.figure(figsize = (10, 5))
sns.violinplot(data = df_tweets,
               x = 'annotation_postPriority',
               y = 'favorite_count',
               order = order_crit,
               color = 'lightblue')
plt.ylim(0, 300)

- For wildfire, earthquake, flood and shooting event types:
The trend seems to be the same than for the retweet counts as the violin plots for each tweet criticality level look almost similar. 

- For typhoon event type:
Favorite counts seem to be more important for critical priority tweets, rather than medium priority tweets which had the highest average retweets.

- For bombing event-types:
Favorite counts seem to be more important for low priority tweets, rather than medium priority tweets which had the highest average retweets.


Let us confirm this intuition by looking at the mean number of favorite for each level of criticality.

In [ ]:
df_tweets[['annotation_postPriority', 'favorite_count']].groupby('annotation_postPriority').mean()

- For wildfire event type:
The same ranking as before can be observed. Critical tweets has a substancially higher favorite count than high tweets, followed by medium and then low tweets.

- For earthquake event type:
We observe that while there are numerous outliers with high, medium and low favorite counts, we observe that critical tweets receive far more favorites on average compared to other criticality types. This is followed by medium and low tweets and finally high criticality tweets where high tweets appear to receive far less favrites on average. This suggests that critical priority tweets are far more loved by the Twitter community in than anything else.

- For flood event type:
We observe that there are many outliers with high, medium and low favorite counts. There are also very little favorites on average for critical criticality tweets. The type of tweet with the most favorites on average is by far the medium priority tweets.

- For flood event type:
We observe that there are many outliers with medium and low favorite counts. There are also very little favorites on average for critical criticality tweets. The type of tweet with the most favorites on average is by far the low priority tweets.

- For typhoon event type:
Favorite counts are greater for critical priority tweets, than any other types of tweets. Many outliers are present for all priority types.

- For bombing event-types:
Favorite counts seem to be far more important for low priority tweets. Many outliers are observed for these types of tweets too.

### 1.3.6 Sentitivity of the tweets

In [ ]:
plt.figure(figsize = (10, 5))
sns.countplot(data = df_tweets,
              x = 'annotation_postPriority',
              hue = 'possibly_sensitive',
              hue_order=[True, False],
              palette = [palette[0], palette[-1]])

- For wildfire event type:
We see here that the relative distribution of possibly sensitive tweets is quite comparable between different levels of criticality. It is important to note that usually possiblibly sensitive tweets are fewer in numbers than 'normal' tweets. However, this is not the case for the low criticality tweets.

- For earthquake event type:
We see here that the relative distribution of possibly sensitive tweets is quite comparable between different levels of criticality. It is important to note that usually possiblibly sensitive tweets are fewer in numbers than 'normal' tweets for all priority types. The gap is smaller for low priority tweets.

- For flood and shooting event type:
We see here that the relative distribution of possibly sensitive tweets is quite comparable between different levels of criticality. It is important to note that usually possiblibly sensitive tweets are fewer in numbers, it is not the case for low priority tweets that have a higher frequency of possibly sensitive tweets here.

- For typhoon event type:
Possibly non sensitive tweets are far better represented for all the criticality tweets types.

- For bombing event-types:
Possibly non sensitive tweets are better represented for all the criticality tweets types, except for critical priority tweets where posibly sensitive tweets slightly edge non sensitive ones

### 1.3.7 Comparing the contents of tweets having different levels of criticality 

In [ ]:
crit = df_tweets[df_tweets.annotation_postPriority == 'Critical'].reset_index()['text'][0]
high = df_tweets[df_tweets.annotation_postPriority == 'High'].reset_index()['text'][0]
medium = df_tweets[df_tweets.annotation_postPriority == 'Medium'].reset_index()['text'][0]
low = df_tweets[df_tweets.annotation_postPriority == 'Low'].reset_index()['text'][0]
print(crit)
print()
print(high)
print()
print(medium)
print()
print(low)

- For wildfire event type:
This example presents interesting distinctions. The critical tweet provides vital information about ongoing events, such as potential solutions to navigate a closed road. The high criticality tweet provides slightly less vital information but it stays relatively informaive. The medium critically tweet is more descriptive than the two previous ones, however there still is some valuable information about the direction of the fire.

Conversely, the low-priority tweet expresses gratitude after the event, directed towards those who participated, which is in stark contrast to the previously described tweets.

This contrast highlights a pattern: high-priority tweets often offer actionable advice to mitigate risks during events, whereas low-priority tweets tend to offer general reflections on the event without specific guidance or recommendations.

- For earthquake event type:
This example presents interesting distinctions. The critical tweet provides good general tips. The high priority tweet provides descriptive updates on the situation. The medium priority tweet provides descriptive information presented in an informal manner. THe low priority tweet provides personal information.

- For flood event type:
The critical and high priority tweets provides descriptive information about infrastructure damage and death toll respectively. THe medium priority tweet shows political developments. Finally, the low priority tweet gives reasurence that rice can survive underwater.

- For shooting event type:
The critical and high priority tweets provides descriptive information about ongoing developments, about the shooter's capture and casualty toll respectively. THe medium priority tweet gives information about both the casulties and the shooters capture. Finally, the low priority tweet gives  a minor descriptive event.

- For typhoon event type:
The critical and high priority tweets provide news updates and information about ongoing developments. The medium criticality tweet gives us a fact about the magnitude of the typhoon. Finally the low criticality tweet is giberish.

- For bombing event type:
The critical and high priority tweets provide breaking news updates and information about ongoing developments. The medium criticality tweet gives us descriptions of the ongoing event. Finally the low criticality tweet is more personal and emotional.

## 2. Embeddings and User-User similarities

### Import the required packages :

In [ ]:
import random

%pip install node2vec
from node2vec import Node2Vec

%pip install gensim
import gensim.downloader
from gensim.models import Word2Vec

from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr

### Select the event to be analyzed, and import the graph 

In [54]:
chosen_event = 'bombing' #can modify this variable to 'wildfire', 'earthquake', 'flood', 'shooting', 'typhoon' or 'bombing'
g = subgraphs.get(chosen_event + '_subgraph')

### 2.1 - Sampling n users

We want to sample n users based on the centralities of the users in the chosen subgraph. 

- We will then take :
    - The 50 more central users based on their degree centrality 
    - The 50 more central users based on their closeness centrality  

- This means we will get between 50 and 100 users sampled


In [ ]:
users = []
nodes_users = []
for node, data in g.nodes(data=True):
    for key, value in data.items():
        if key == 'labels':
            if value == ':User':
                nodes_users.append(node)
                users.append({node : data})

#We choose the size of the sample
n=100

# Sample n/2 users with highest degree centrality
degree_centralities = nx.degree_centrality(g) #We first compute the degree coef fo all nodes
degree_filtred = {key: value for key, value in degree_centralities.items() if key in nodes_users} #We then only select the users nodes
top_degree_users = sorted(degree_filtred, key=lambda x: degree_filtred[x], reverse=True)[0:int(n/2)] #We take the 50 users that have the highest degree centrality


filtered_nodes = [node for node in nodes_users if node not in top_degree_users] #We create a list of all the users except the ones already in the top 50 of degree centrality

# Sample n/2 users with highest closeness centrality (taking away the nodes that are already in the highest degree)
closeness_centralities = nx.closeness_centrality(g) #We first compute the closeness coef fo all nodes
closeness_filtred = {key: value for key, value in closeness_centralities.items() if key in filtered_nodes} #We then only select the users nodes (that are not in the top 50 degree centrality)
top_closeness_users = sorted(closeness_filtred, key=lambda x: closeness_filtred[x], reverse=True)[0:int(n/2)]  #We take the 50 users that have the highest closeness centrality

# List of the selected users
list_user = list(set(top_degree_users + top_closeness_users))

users_sample = []

for node_dict in users:
    node_key = list(node_dict.keys())[0]  
    if node_key in list_user:  
        users_sample.append({node_key: node_dict[node_key]})

# exemple of user 
users_sample[0]

## 2.ii.a - Embeddings on Graph Structure

Let us create the subgraph containing our sampled users

In [56]:
# first, we need the associated nodes ids to the sample we definded : 
node_sample_ids = []
for i in range(len(users_sample)):
    for key, _ in users_sample[i].items():
        node_sample_ids.append(key)

# define the subgraph with the sample of 100 nodes
sample_graph = g.subgraph(node_sample_ids)

We now need to fit a Node2Vec model on our sub-subgraph, that will create embeddings from the graph structure

In [57]:
# we can then build the embeddings of the sampled graph with Node2Vec
node2vec = Node2Vec(sample_graph, dimensions=50)
fitted_model_n2v = node2vec.fit(window=10, min_count=1) 

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:00<00:00, 245.31it/s]


We can have access to the 10 most similar nodes of a node, based on cosine similarity

In [ ]:
for key, value in users_sample[0].items():
    k = key
fitted_model_n2v.wv.most_similar(k)

Or we can build a cosine similarity matrix :

In [59]:
# get the embeddings for every node
node_embeddings_n2v = {node: fitted_model_n2v.wv[node] for node in sample_graph.nodes()}

# list element is easier to handle
list_of_embeddings_n2v = []
for key, value in node_embeddings_n2v.items():
    list_of_embeddings_n2v.append({key : value})

In [60]:
# build the cosine similarity matrix
cos_sim_n2v = [[0 for _ in range(100)] for _ in range(100)]

for i, emb_i_dict in enumerate(list_of_embeddings_n2v):
    for j, emb_j_dict in enumerate(list_of_embeddings_n2v):
        emb_i = next(iter(emb_i_dict.values())) 
        emb_j = next(iter(emb_j_dict.values()))

        cosine_sim = cosine_similarity([emb_i], [emb_j])[0][0]
        cos_sim_n2v[i][j] = cosine_sim

With this cosine similarity matrix, we can easily get a list of highly similar users :

In [ ]:
# from this, we can get the more similar users in our sample graph :
arr_cos_sim_n2v = np.array(cos_sim_n2v)
np.fill_diagonal(arr_cos_sim_n2v, -np.inf) # we replace the 1 elements of the diagonal by -inf

# 10 most similar nodes
v = []

nb_users_to_print = 10
for _ in range(nb_users_to_print):
    max_index = np.argmax(arr_cos_sim_n2v)
    max_sim = np.max(arr_cos_sim_n2v)

    max_row_index, max_col_index = np.unravel_index(max_index, np.array(arr_cos_sim_n2v).shape)

    arr_cos_sim_n2v[max_row_index][max_col_index] = -np.inf
    arr_cos_sim_n2v[max_col_index][max_row_index] = -np.inf

    v.append((max_row_index, max_col_index, max_sim))

print(f"The {nb_users_to_print} most similar pairs of users are :")
v

## 2.ii.b - Embeddings on Post Content

Let us create a list of tweets containing the tweets that were posted by the users in our previously created subgraph

In [ ]:
# we already have our sampled graph and the users, we now need the tweets
tweets = []
for node, data in g.nodes(data=True):
    for key, value in data.items():
        if key == 'labels':
            if value == ':Tweet':
                tweets.append({node : data})

# exemple of tweet
tweets[0]

We will need a list of dictionnaries containing the user_id as keys, and their posted tweets as values

In [ ]:
# build a list containing dictionnaries with the user_node_id and every tweet made by this user
users_posts = []

for u in users_sample:
    user_node_id = [key for key, _ in u.items()][0]
    tweets_by_user = []
    
    for t in tweets:
        tweet_node_id = [key for key, _ in t.items()][0]
        text_tweet = [value for _, value in t.items()][0]['text']

        if tweet_node_id in g[user_node_id]:
            tweets_by_user.append(text_tweet)

    users_posts.append({'user':user_node_id,
                        'tweets':tweets_by_user})
    
# exemple of users posts :
users_posts[:2]

Let us load a pretrained Word2Vec model on tweeter data, that will convert the posts by users into embeddings

In [6]:
# let us build the empbeddings of our tweets, using a pretrained Word2Vec model on twitter data, using embeddings of length 50 to match the graph structure embeddings
w2v = gensim.downloader.load('glove-twitter-50')

We also need a function that will create the embedding of a given tweet

In [65]:
# we define a function that allows us to tokenize a tweet/sentence, and we take the average of each embedding
def sentence_embedding(sentence):
    if len(sentence)>0:
        tokens = sentence.split()
        embeddings = [w2v[token] for token in tokens if token in w2v]
        
        if len(tokens) > 0 and embeddings:
                avg_embedding = np.mean(embeddings, axis=0)
        else:
            avg_embedding = np.zeros(w2v.vector_size)
    else:
         avg_embedding = np.zeros(w2v.vector_size)
    return avg_embedding

Let us compute the embeddings of our set of tweets and add them to the list of dictionnaries

In [ ]:
# we then compute and add the embedded tweets to the users_posts list, by adding a key to each dictionnary in the list
for i in users_posts:
    tweets = i['tweets']
    embedded_tweets = []

    if len(tweets) == 0:
        avg_emb_tweet = np.zeros(w2v.vector_size)

    else :
        for j in i['tweets']:
            emb_tweet = sentence_embedding(j)
            embedded_tweets.append(emb_tweet)
            avg_emb_tweet = np.mean(embedded_tweets, axis=0)
    i['embedded_tweets'] = avg_emb_tweet

# exemple of new user post :
users_posts[0]

We now can create a cosine similarity matrxi based on our embedded tweets

In [67]:
# we can now compute the cosine similarity matrix like with our graph structure embeddings
cos_sim_w2v = [[0 for _ in range(100)] for _ in range(100)]

for i in range(len(users_posts)):
    for j in range(len(users_posts)):
        if i == j:
            cos_sim_w2v[i][j] = 1
        else:
            emb_i = users_posts[i]['embedded_tweets'].reshape(1, -1)
            emb_j = users_posts[j]['embedded_tweets'].reshape(1, -1)
            cos_sim_w2v[i][j] = cosine_similarity(emb_i, emb_j)[0][0]

Now, we can get a list of very similar users

In [ ]:
# finally, we can print the most similar users based on this matrix
arr_cos_sim_w2v = np.array(cos_sim_w2v)
np.fill_diagonal(arr_cos_sim_w2v, -np.inf) # replace the diagonal full of 1s by -inf 

v = []

nb_users_to_print = 10
for _ in range(nb_users_to_print):
    max_index = np.argmax(arr_cos_sim_w2v)
    max_sim = np.max(arr_cos_sim_w2v)

    max_row_index, max_col_index = np.unravel_index(max_index, np.array(arr_cos_sim_w2v).shape)

    arr_cos_sim_w2v[max_row_index][max_col_index] = -np.inf
    arr_cos_sim_w2v[max_col_index][max_row_index] = -np.inf
    
    v.append((max_row_index, max_col_index, max_sim))

print(f"The {nb_users_to_print} more similar pairs of users are :")
v

In [ ]:
# exemple of tweets, we can compare them 
print(users_posts[v[1][0]]['tweets'])
print()
print(users_posts[v[1][1]]['tweets'])

For the earthquake subgraph for example, it seems that both most similar tweets are talking mainly about events that happened in Nepal

## 2.iii - Trends in Correlations

Let us define a general function that summarizes all the steps we just realized, to be looped for all event types

- This function will use the Word2Vec pretrained model that we loaded before, so make sure you have loaded it

In [70]:
def similarity_matrix(selected_graph):

    g = subgraphs.get(selected_graph + '_subgraph')

    users = []
    nodes_users = []
    for node, data in g.nodes(data=True):
        for key, value in data.items():
            if key == 'labels':
                if value == ':User':
                    nodes_users.append(node)
                    users.append({node : data})

    #We choose the size of the sample
    n=100

    # Sample n/2 users with highest degree centrality
    degree_centralities = nx.degree_centrality(g) #We first compute the degree coef fo all nodes
    degree_filtred = {key: value for key, value in degree_centralities.items() if key in nodes_users} #We then only select the users nodes
    top_degree_users = sorted(degree_filtred, key=lambda x: degree_filtred[x], reverse=True)[0:int(n/2)] #We take the 50 users that have the highest degree centrality


    filtered_nodes = [node for node in nodes_users if node not in top_degree_users] #We create a list of all the users except the ones already in the top 50 of degree centrality

    # Sample n/2 users with highest closeness centrality (taking away the nodes that are already in the highest degree)
    closeness_centralities = nx.closeness_centrality(g) #We first compute the closeness coef fo all nodes
    closeness_filtred = {key: value for key, value in closeness_centralities.items() if key in filtered_nodes} #We then only select the users nodes (that are not in the top 50 degree centrality)
    top_closeness_users = sorted(closeness_filtred, key=lambda x: closeness_filtred[x], reverse=True)[0:int(n/2)]  #We take the 50 users that have the highest closeness centrality

    # List of the selected users
    list_user = list(set(top_degree_users + top_closeness_users))

    users_sample = []

    for node_dict in users:
        node_key = list(node_dict.keys())[0]  
        if node_key in list_user:  
            users_sample.append({node_key: node_dict[node_key]})


    # first, we need the associated nodes ids to the sample we definded : 
    node_sample_ids = []
    for i in range(len(users_sample)):
        for key, _ in users_sample[i].items():
            node_sample_ids.append(key)

    # define the subgraph with the sample of 100 nodes
    sample_graph = g.subgraph(node_sample_ids)

    # we can then build the embeddings of the sampled graph with Node2Vec
    node2vec = Node2Vec(sample_graph, dimensions=50)
    fitted_model_n2v = node2vec.fit(window=10, min_count=1) 
    
    # get the embeddings for every node
    node_embeddings_n2v = {node: fitted_model_n2v.wv[node] for node in sample_graph.nodes()}

    # list element is easier to handle
    list_of_embeddings_n2v = []
    for key, value in node_embeddings_n2v.items():
        list_of_embeddings_n2v.append({key : value})

    # build the cosine similarity matrix
    cos_sim_n2v = [[0 for _ in range(100)] for _ in range(100)]

    for i, emb_i_dict in enumerate(list_of_embeddings_n2v):
        for j, emb_j_dict in enumerate(list_of_embeddings_n2v):
            emb_i = next(iter(emb_i_dict.values())) 
            emb_j = next(iter(emb_j_dict.values()))

            cosine_sim = cosine_similarity([emb_i], [emb_j])[0][0]
            cos_sim_n2v[i][j] = cosine_sim


    # we already have our sampled graph and the users, we now need the tweets
    tweets = []
    for node, data in g.nodes(data=True):
        for key, value in data.items():
            if key == 'labels':
                if value == ':Tweet':
                    tweets.append({node : data})


    # build a list containing dictionnaries with the user_node_id and every tweet made by this user
    users_posts = []

    for u in users_sample:
        user_node_id = [key for key, _ in u.items()][0]
        tweets_by_user = []
        
        for t in tweets:
            tweet_node_id = [key for key, _ in t.items()][0]
            text_tweet = [value for _, value in t.items()][0]['text']

            if tweet_node_id in g[user_node_id]:
                tweets_by_user.append(text_tweet)

        users_posts.append({'user':user_node_id,
                            'tweets':tweets_by_user})    


    # we then compute and add the embedded tweets to the users_posts list, by adding a key to each dictionnary in the list
    for i in users_posts:
        tweets = i['tweets']
        embedded_tweets = []

        if len(tweets) == 0:
            avg_emb_tweet = np.zeros(w2v.vector_size)

        else :
            for j in i['tweets']:
                emb_tweet = sentence_embedding(j)
                embedded_tweets.append(emb_tweet)
                avg_emb_tweet = np.mean(embedded_tweets, axis=0)
        i['embedded_tweets'] = avg_emb_tweet

    # we can now compute the cosine similarity matrix like with our graph structure embeddings
    cos_sim_w2v = [[0 for _ in range(100)] for _ in range(100)]

    for i in range(len(users_posts)):
        for j in range(len(users_posts)):
            if i == j:
                cos_sim_w2v[i][j] = 1
            else:
                emb_i = users_posts[i]['embedded_tweets'].reshape(1, -1)
                emb_j = users_posts[j]['embedded_tweets'].reshape(1, -1)
                cos_sim_w2v[i][j] = cosine_similarity(emb_i, emb_j)[0][0]
    return([cos_sim_n2v, cos_sim_w2v])

Loop the function through all the subgraphs

In [ ]:
correlation_results = []

events = ['wildfire', 'earthquake', 'typhoon', 'bombing', 'flood', 'shooting']

for event_type in events:

    res = similarity_matrix(event_type)
    
    graph_similarities = [similarity for sublist in res[0] for similarity in sublist]
    content_similarities = [similarity for sublist in res[1] for similarity in sublist]

    # Compute the correlation between the graph structure and tweet content similarities
    correlation, p_value = spearmanr(graph_similarities, content_similarities)
    
    # Append the correlation coefficient and p-value to the results list
    correlation_results.append({'event_type': event_type, 'correlation': correlation, 'p_value': p_value})

In [ ]:
correlation_results

According to this results, we see that the similarity matrices for the graph structure and post contents are more correlated for the flood and shooting event (higher p-value) than for the earthquake and typhoon events. Moreover we see that in general we observe a positive correlation between the two matrices of similarity except for the shooting event.

## 3. Information Retrieval System

Let us import the nltk package that we will need for stopwords management

In [ ]:
%pip install nltk
import nltk
from nltk.corpus import stopwords

Let us re-import the graph to be sure we have the correct file

In [7]:
path = 'database_formated_for_NetworkX.graphml'
graph = nx.read_graphml(path)

We first need the users and tweets of our graph

In [ ]:
# retrieve the users of our graph
users = []
nodes_users = []
for node, data in graph.nodes(data=True):
    for key, value in data.items():
        if key == 'labels':
            if value == ':User':
                nodes_users.append(node)
                users.append({node : data})

# exemple of user
print(users[0])

# retrieve the tweets of our graph
tweets = []
for node, data in graph.nodes(data=True):
    for key, value in data.items():
        if key == 'labels':
            if value == ':Tweet':
                tweets.append({node : data['text']})

# exemple of tweet
print(tweets[1])

First, let us build a list containing the Poster of a tweet, his tweet, and the degree centrality of the poster

In [ ]:
tweets_and_poster = []
list_centralities = nx.degree_centrality(graph)

for t in tweets:
    for key, value in t.items():
        tweet_id = key
    neighb = list(graph.neighbors(tweet_id))
    for n in neighb:
        if graph.nodes[n]['labels'] == ':User':
            tweets_and_poster.append({'poster' : n,
                                      'tweet' : value,
                                      'deg_centrality' : list_centralities[n]})

# exmple 
tweets_and_poster[0]

Remove the stopwords from all of our tweets

In [ ]:
# import english stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# function to remove the stopwords from a tweet
def remove_stopwords(tweet):
    words = tweet.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [ ]:
# remove stopwords from all the tweets that we consider
for t in tweets_and_poster:
    t['tweet'] = remove_stopwords(t['tweet'])

# exemple of tweets without stopwords
tweets_and_poster[0]

Now, let us build a function to get the embeddings of each tweet

We will use the Word2Vec model that we previously loaded for question 2.ii.b

In [14]:
def embeddings(tweet_and_poster):
    tweet = tweet_and_poster['tweet']
    if len(tweet)>0:
        tokens = tweet.split()
        embeddings = [w2v[token] for token in tokens if token in w2v]
        
        if len(tokens) > 0 and embeddings:
                avg_embedding = np.mean(embeddings, axis=0)
        else:
            avg_embedding = np.zeros(w2v.vector_size)
    else:
         avg_embedding = np.zeros(w2v.vector_size)

    return avg_embedding

We compute the weighted embeddings of our tweets and add them to our *tweets_and_poster* list

In [ ]:
# add the weighted embedding key to the tweet and poster list of dictionnaries
for t in tweets_and_poster:
    t['weighted_embedding'] = embeddings(t)

# exemple of embedding
tweets_and_poster[0]

Now, let us build the top-k tweets retrieval of a query

- Since the query is supposed to be built of keywords, it should not contain stopwords

- If we compute the cosine similarity of the embedded query and all the embeddings, we can retrieve the top-k more relevant tweets

- We can also account for posters centrality by multiplying the obtained cosine similarities by the degree centrality of the related poster

In [16]:
def top_k_retrieval(keywords, k):
    print(keywords)
    print() 

    embedding = [w2v[token] for token in keywords if token in w2v]
    embedding = np.mean(embedding, axis=0).reshape(1, -1)

    similarity_list = [cosine_similarity(embedding, i['weighted_embedding'].reshape(1, -1)) for i in tweets_and_poster]
    weighted_similarity_list = [similarity_list[i] * tweets_and_poster[i]['deg_centrality']for i in range(len(similarity_list))]

    related_tweets = [i['tweet'] for i in tweets_and_poster]

    sorted_weighted_similarity_list = sorted(zip(weighted_similarity_list, related_tweets), reverse = True)
    sorted_sims, sorted_tweets = zip(*sorted_weighted_similarity_list)

    for i in range(len(sorted_sims[:k])):
        print(f'Cosine Sim : {sorted_sims[i][0][0]} \n Tweet : {sorted_tweets[i]} \n')

In [ ]:
query = ['drive', 'fire', 'hospital', 'injury']
k = 10

top_k_retrieval(query, k)